## Start

In [1]:
"""
Inference Script
"""
import json
import os
import torch
import tqdm
from shutil import copyfile


# Detectron imports|
from detectron2.engine import launch
from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.data import *
from detectron2.data.build import trivial_batch_collator
from detectron2.data.samplers.distributed_sampler import InferenceSampler

# Project imports
import core.datasets.metadata as metadata

from core.setup import setup_config, setup_arg_parser
from offline_evaluation import compute_average_precision, compute_probabilistic_metrics, compute_calibration_errors
from probabilistic_inference.probabilistic_inference import build_predictor
from probabilistic_inference.inference_utils import instances_to_json

from baal.active import FileDataset, ActiveLearningDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
##setup inference args, this also contains all the training args
arg_parser = setup_arg_parser()
args = arg_parser.parse_args("")
# Support single gpu inference only.
args.num_gpus = 1
args.dataset_dir = '/public-dataset/BDD/bdd100k'
args.test_dataset = 'bdd_val'
args.config_file = '/home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml'
args.inference_config = '/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml'
print("Command Line Args:", args)

Command Line Args: Namespace(config_file='/home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml', dataset_dir='/public-dataset/BDD/bdd100k', dist_url='tcp://127.0.0.1:50162', eval_only=False, inference_config='/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml', iou_correct=0.7, iou_min=0.1, machine_rank=0, min_allowed_score=0.0, num_gpus=1, num_machines=1, opts=[], random_seed=0, resume=False, test_dataset='bdd_val')


In [3]:
global result_list

In [4]:
#method inside the build detection train loader that loads from config
def load_dataset_and_mapper_from_config(cfg, dataset_name='bdd_val', mapper=None):
    """
    Uses the given `dataset_name` argument (instead of the names in cfg), because the
    standard practice is to evaluate each test set individually (not combining them).
    """
    dataset = get_detection_dataset_dicts(
        [dataset_name],
        filter_empty=False,
        proposal_files=[
            cfg.DATASETS.PROPOSAL_FILES_TEST[list(cfg.DATASETS.TEST).index(dataset_name)]
        ]
        if cfg.MODEL.LOAD_PROPOSALS
        else None,
    )
    # use active learning dataset wrapper
    if mapper is None:
        mapper = DatasetMapper(cfg, False)
        
    return dataset, mapper
    
    

In [5]:
def dataloader_from_config(cfg, dataset, mapper):
    if isinstance(dataset, list):
        dataset = DatasetFromList(dataset, copy=False)
    if mapper is not None:
        dataset = MapDataset(dataset, mapper)
    sampler = InferenceSampler(len(dataset))
    # Always use 1 image per worker during inference since this is the
    # standard when reporting inference time in papers.
    batch_sampler = torch.utils.data.sampler.BatchSampler(sampler, 1, drop_last=False)
    num_worker = 4
    data_loader = torch.utils.data.DataLoader(
        dataset,
        num_workers=num_worker,
        batch_sampler=batch_sampler,
        collate_fn=trivial_batch_collator,
    )
    return data_loader

In [6]:
#method inside the build detection train loader that loads from config
def test_loader_from_config(cfg, dataset_name='bdd_val', mapper=None):
    """
    Uses the given `dataset_name` argument (instead of the names in cfg), because the
    standard practice is to evaluate each test set individually (not combining them).
    """
    dataset = get_detection_dataset_dicts(
        [dataset_name],
        filter_empty=False,
        proposal_files=[
            cfg.DATASETS.PROPOSAL_FILES_TEST[list(cfg.DATASETS.TEST).index(dataset_name)]
        ]
        if cfg.MODEL.LOAD_PROPOSALS
        else None,
    )
    # use active learning dataset wrapper
    dataset = ActiveLearningDataset(dataset)
    dataset.label_randomly(100)
    if mapper is None:
        mapper = DatasetMapper(cfg, False)
    
    if isinstance(dataset, list):
        dataset = DatasetFromList(dataset, copy=False)
    if mapper is not None:
        dataset = MapDataset(dataset, mapper)
    sampler = InferenceSampler(len(dataset))
    # Always use 1 image per worker during inference since this is the
    # standard when reporting inference time in papers.
    batch_sampler = torch.utils.data.sampler.BatchSampler(sampler, 1, drop_last=False)
    num_worker = 4
    data_loader = torch.utils.data.DataLoader(
        dataset,
        num_workers=num_worker,
        batch_sampler=batch_sampler,
        collate_fn=trivial_batch_collator,
    )
    return data_loader

In [7]:
# run this once per session only
#cfg = setup_config(args, random_seed=args.random_seed, is_testing=True)

In [8]:
cfg = setup_config(args, random_seed=args.random_seed, is_testing=True)

# Make sure only 1 data point is processed at a time. This simulates
# deployment.
cfg.defrost()
cfg.DATALOADER.NUM_WORKERS = 32
cfg.SOLVER.IMS_PER_BATCH = 1

cfg.MODEL.DEVICE = device.type

# Set up number of cpu threads
torch.set_num_threads(cfg.DATALOADER.NUM_WORKERS)

# Create inference output directory and copy inference config file to keep
# track of experimental settings
inference_output_dir = os.path.join(
    cfg['OUTPUT_DIR'],
    'inference',
    args.test_dataset,
    os.path.split(args.inference_config)[-1][:-5])
os.makedirs(inference_output_dir, exist_ok=True)
copyfile(args.inference_config, os.path.join(
    inference_output_dir, os.path.split(args.inference_config)[-1]))

# Get category mapping dictionary:
train_thing_dataset_id_to_contiguous_id = MetadataCatalog.get(
    cfg.DATASETS.TRAIN[0]).thing_dataset_id_to_contiguous_id
test_thing_dataset_id_to_contiguous_id = MetadataCatalog.get(
    args.test_dataset).thing_dataset_id_to_contiguous_id

# If both dicts are equal or if we are performing out of distribution
# detection, just flip the test dict.
if (train_thing_dataset_id_to_contiguous_id == test_thing_dataset_id_to_contiguous_id) or (
        cfg.DATASETS.TRAIN[0] == 'coco_not_in_voc_2017_train'):
    cat_mapping_dict = dict(
        (v, k) for k, v in test_thing_dataset_id_to_contiguous_id.items())
else:
    # If not equal, two situations: 1) BDD to KITTI and 2) COCO to PASCAL
    cat_mapping_dict = dict(
        (v, k) for k, v in test_thing_dataset_id_to_contiguous_id.items())
    if 'voc' in args.test_dataset and 'coco' in cfg.DATASETS.TRAIN[0]:
        dataset_mapping_dict = dict(
            (v, k) for k, v in metadata.COCO_TO_VOC_CONTIGUOUS_ID.items())
    elif 'kitti' in args.test_dataset and 'bdd' in cfg.DATASETS.TRAIN[0]:
        dataset_mapping_dict = dict(
            (v, k) for k, v in metadata.BDD_TO_KITTI_CONTIGUOUS_ID.items())
    else:
        ValueError(
            'Cannot generate category mapping dictionary. Please check if training and inference datasets are compatible.')
    cat_mapping_dict = dict(
        (dataset_mapping_dict[k], v) for k, v in cat_mapping_dict.items())

# Build predictor
predictor = build_predictor(cfg)
#orig_test_data_loader = build_detection_test_loader(cfg, dataset_name=args.test_dataset)
#test_data_loader = test_loader_from_config(cfg, dataset_name='bdd_val', mapper=None)

dataset1, mapper = load_dataset_and_mapper_from_config(cfg, dataset_name='bdd_val', mapper=None)

dataset = ActiveLearningDataset(dataset)
dataset.label([1,2,3])
#dataset.label_randomly(100)

test_data_loader = dataloader_from_config(cfg, dataset, mapper)

final_output_list = []
if not args.eval_only:
    with torch.no_grad():
        with tqdm.tqdm(total=len(test_data_loader)) as pbar:
            for idx, input_im in enumerate(test_data_loader):
                #print(input_im.size)
                outputs = predictor(input_im)

                final_output_list.extend(
                    instances_to_json(
                        outputs,
                        input_im[0]['image_id'],
                        cat_mapping_dict))
                pbar.update(1)
                result_list = outputs
                break

    with open(os.path.join(inference_output_dir, 'coco_instances_results.json'), 'w') as fp:
        json.dump(final_output_list, fp, indent=4,
                  separators=(',', ': '))

compute_average_precision.main(args, cfg)
#compute_probabilistic_metrics.main(args, cfg)
#compute_calibration_errors.main(args, cfg)

Loading config /home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/../../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
Config '/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml' has no VERSION. Assuming it to be compatible with latest v2.


[02/21 13:37:39 detectron2]: Rank of current process: 0. World size: 1
[02/21 13:37:40 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------------
sys.platform            linux
Python                  3.8.5 (default, Sep  4 2020, 07:30:14) [GCC 7.3.0]
numpy                   1.19.5
detectron2              0.3 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 10.2
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.7.1 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/torch
PyTorch debug build     False
GPU available           True
GPU 0                   Tesla V100-SXM2-32GB (arch=7.0)
CUDA_HOME               /usr/local/cuda
Pillow                  8.1.0
torchvision             0.8.2 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/torchvision
torchvision arch flags  3.5, 5.0

[02/21 13:37:40 detectron2]: Full config saved to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_0/config.yaml
[02/21 13:37:44 d2.data.datasets.coco]: Loaded 10000 images in COCO format from /public-dataset/BDD/bdd100k/labels/val_coco_format.json
[02/21 13:37:45 d2.data.build]: Distribution of instances among all 7 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    car     | 102506       |    bus     | 1597         |   truck    | 4245         |
|   person   | 13262        |   rider    | 649          |    bike    | 1007         |
|   motor    | 452          |            |              |            |              |
|   total    | 123718       |            |              |            |              |
[02/21 13:37:45 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in in

NameError: name 'dataset' is not defined

In [11]:
test_data_loader.dataset

In [15]:
print(dataset.)

AttributeError: 'ActiveLearningDataset' object has no attribute 'dataset'

In [13]:
dataset[1:20]

ValueError: can only convert an array of size 1 to a Python scalar

In [9]:
dataset1, mapper = load_dataset_and_mapper_from_config(cfg, dataset_name='bdd_val', mapper=None)

[02/21 13:37:51 d2.data.datasets.coco]: Loaded 10000 images in COCO format from /public-dataset/BDD/bdd100k/labels/val_coco_format.json
[02/21 13:37:52 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]


In [19]:
dataset1[1:10][0]['file_name']

'/public-dataset/BDD/bdd100k/images/100k/val/b1c81faa-3df17267.jpg'

In [11]:
data1 = dataset1[:50]
data2 = dataset1[50:]

In [12]:
import torch.multiprocessing as mp

In [13]:
def data_process(name_list, dataset):
    for data in dataset:
        name_list.append(data['file_name'])


In [29]:
name_list = []

data_process(name_list, data1)

In [30]:
name_list

['/public-dataset/BDD/bdd100k/images/100k/val/b1c66a42-6f7d68ca.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1c81faa-3df17267.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1c81faa-c80764c5.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1c9c847-3bda4659.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1ca2e5d-84cf9134.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1ca8418-84a133a0.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1cac6a7-04e33135.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1cd1e94-26dd524f.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1cd1e94-549d0bfe.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1ceb32e-3f481b43.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1ceb32e-51852abe.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1ceb32e-813f84b2.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1ceb32e-a106591d.jpg',
 '/public-dataset/BDD/bdd100k/images/100k/val/b1cebfb7-284f5117.jpg',
 '/public-dataset/BD

In [14]:
num_processes = 2
list1 = []
list2 = []

group_list = [list1, list2]
data_list = [data1, data2]

processes = []

#mp.set_start_method('spawn')

for rank in range(num_processes):
    print(rank)
    p = mp.Process(target=data_process, args=(group_list[rank],data_list[rank]))
    p.start()
    processes.append(p)
    
for p in processes:
    p.join()

0
1


In [39]:
group_list

[[], []]

In [41]:
data_list[0]

[{'file_name': '/public-dataset/BDD/bdd100k/images/100k/val/b1c66a42-6f7d68ca.jpg',
  'height': 720,
  'width': 1280,
  'image_id': 0,
  'annotations': [{'iscrowd': 0,
    'bbox': [205.879417, 338.141346, 76.11299600000001, 49.910161000000016],
    'category_id': 0,
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'iscrowd': 0,
    'bbox': [47.414654, 344.380116, 82.35176600000001, 56.148933],
    'category_id': 0,
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'iscrowd': 0,
    'bbox': [247.0553, 344.380116, 99.82032400000003, 51.157916],
    'category_id': 0,
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'iscrowd': 0,
    'bbox': [0, 336.893592, 52.40567, 66.130964],
    'category_id': 0,
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'iscrowd': 0,
    'bbox': [650.079856, 354.362148, 13.725295000000074, 27.450589000000036],
    'category_id': 4,
    'bbox_mode': <BoxMode.XYWH_ABS: 1>},
   {'iscrowd': 0,
    'bbox': [648.832101, 366.83969, 14.973050000000057, 21.211816999999996],
    'categor

In [ ]:
orig_test_data_loader = build_detection_test_loader(cfg, dataset_name=args.test_dataset)

In [1]:
import concurrent.futures
import time

In [13]:
def do_something(a, b):
    a = np.array([a for _ in range(3)])
    print(f'Sleeping a+b = {c} second(s)...')
    return {'a': a, 'b': b, 'c': c}

In [3]:
processes = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    p1 = executor.submit(do_something, 1, 2)
    p2 = executor.submit(do_something, 2, 3)
    processes.append(p1)
    processes.append(p2)
    
    for process in processes:
        print(process.result())
    

Sleeping a+b = 3 second(s)...Sleeping a+b = 5 second(s)...

Done Sleeping...3
Done Sleeping...5


In [4]:
tup = (1,2)

In [5]:
a = [1, 2]
b = [2, 3]

arg = list(zip(a,b))
print(arg)

[(1, 2), (2, 3)]


In [6]:
a, b = tup
print(a)
print(b)
print(tup)

1
2
(1, 2)


In [11]:
a = 1
b = [2, 3]
a_s = [a for _ in range(len(b))]


arg = [a, b]

result_list_a = []
result_list_b = []
result_list_c = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(do_something, a_s, b)
    for result in results:
        print(result)
        result_list.extend(result)
        

Sleeping a+b = 3 second(s)...Sleeping a+b = 4 second(s)...

[1, 2, 3]
[1, 3, 4]


In [12]:
print(result_list)

[1, 2, 3, 1, 3, 4]


In [9]:
result_list

['Done Sleeping...3', 'Done Sleeping...4']